<a href="https://colab.research.google.com/github/daddyawesome/writings/blob/master/ETLExcel_to_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# From Excel To Databases with Python

As a business analyst, not a day goes by that I do not find myself fiddling with some data in an excel spreadsheet. Talk to anyone working in an analytical role, and they will tell you about their love-hate relationship with excel. For all the good things that excel can do; it’s simply a pain to work with when it comes to larger data sets.   
<br>
Pivots take forever to load, the machine runs out of memory, and before you know it, the whole thing becomes unmanageable. Not to mention that excel can only support up to 1,048,576 rows. Sure, you could consider doing things in VBA, but what’s the point.
<br>
If only there were an easy way to transfer data into a SQL Database, do your analysis and then delete it all. Well, this is where Python swoops in to save the day.
<br>
## SQL In Python
To begin with, let us explore the most popular options when it comes to SQL in Python. The two most popular SQL DBs to work within Python is MySQL and SQLite.
MySQL has two popular libraries associated with it: PyMySQL and MySQLDb; while SQLite has SQLite3.   
<br>
SQLite is what is known as an embedded database, which means it runs within our application and hence it is not required to be installed somewhere first (unlike MySQL).   
<br>
This is a significant difference; and pivotal in our quest for quick data analysis. As such, we will go ahead and learn how to use SQLite.

## Setting up SQLite in Python
The first thing we need to do is import the library:

In [ ]:
import sqlite3

Then we need to determine whether we would like to save this database anywhere or simply hold it in memory while our application is running.   
<br>
If decided to actually save the database down with any of the data imported, we would then have to give the DB a name, say ‘FinanceExplainedDb’, and have the following command:

In [ ]:
dbname = 'FinanceExplainedDb'
conn = sqlite3.connect(dbname + '.sqlite')

On the other hand, if we wanted the whole thing in memory, and for it to vanish when we were done, we could use the following command:

In [ ]:
conn = sqlite3.connect(':memory:')

At this point, SQLite is all set up and ready to be used in Python. Assuming we had some data loaded in the DB under `Table1`, we could execute SQL commands in the following way:

In [ ]:
cur = conn.cursor()
cur.execute('SELECT * FROM Table1')
for row in cur:
    print(row)

Let us now explore how we can make our data available through our application using Pandas.

## Using Pandas to load data in our application
Assuming that we already have the data, we would like to analyse, we can use the Python Pandas library to do it.   
<br>
First, we need to import the Pandas library and then we can load the data in a data frame (You can think of data frames as an array of sorts):

In [ ]:
import pandas as pd
#if we have a csv file
df = pd.read_csv('ourfile.csv')
#if we have an excel file
df = pd.read_excel('ourfile.xlsx')

Once we have loaded the data, we can put it straight into our SQL Database with a simple command:

In [ ]:
df.to_sql(name='Table1', con=conn)

If you are loading multiple files within the same table, you can use the `if_exists` parameter:

In [ ]:
df.to_sql(name='Table1', con=conn, if_exists='append')

## Memory Considerations
When it comes to dealing with larger data sets, we will not be able to use this one-line command to load the data. Our application will run out of memory.   
<br>
Instead, we will have to load our data little by little.  
<br>
For this example, let’s assume we will load 10,000 rows at a time:

In [ ]:
chunksize = 10000
for chunk in pd.read_csv('ourfile.csv', chunksize=chunksize):
    chunk.to_sql(name='Table1', con=conn, if_exists='append')

## Bringing it all together
To bring everything together, I have decided to give you a Python script that covers most of the things we talked about.   

The script will do the following things:
- Load some sample data from a Python library
- Write the data out to a CSV
- Load the data back into our application through the CSV in a data frame chunk by chunk and put in a DB
- Then execute a SELECT statement on the database

In [ ]:
import sqlite3, pandas as pd, numpy as np
#####Creating test data for us -- you can ignore
from sklearn import datasets
iris = datasets.load_iris()
df1 = pd.DataFrame(data= np.c_[iris['data'], iris['target']], columns= iris['feature_names'] + ['target'])
df1.to_csv('TestData.csv',index=False)
###########################
conn = sqlite3.connect(':memory:')
cur = conn.cursor()
chunksize = 10
for chunk in pd.read_csv('TestData.csv', chunksize=chunksize):
    chunk.columns = chunk.columns.str.replace(' ', '_') #replacing spaces with underscores for column names
    chunk.to_sql(name='Table1', con=conn, if_exists='append')
cur.execute('SELECT * FROM Table1')
names = list(map(lambda x: x[0], cur.description)) #Returns the column names
print(names)
for row in cur:
    print(row)
cur.close()

There you have it guys — a brief introduction on how you can import your data into a database for quick analysis.